In [1]:
%run Latex_macros.ipynb

<IPython.core.display.Latex object>

# Pre-train, prompt, predict 

If you expect a "raw" LLM (e.g., GPT-3) to behave like ChatGPT: you will be disappointed.
- the LLM is trained to "predict the next" token
    - continue the text given in the prompt
- ChatGPT  is a raw LLM that has been Fine-Tuned to be
    - A Helpful Assistant: answer questions

There *are* ways to adapt the raw LLM to be more like ChatGPT without Fine-Tuning.

Respect the LLM training objective.  
- Turn your questions into instances of "text continuation"
    - the Universal API
- Use In-Context Learning: exemplars that illustrate the behavior of a Helpful Assistant

Here are suggestions on how to get the "expected behavior" out of the LLaMa LLM:

<br>
<table>
    <center><strong>Prompts to get a raw LLM to generate helpful results</strong></center>
    <img src="images/Llama_guide_to_good_generating.png">
    <br>
    Attribution: https://github.com/facebookresearch/llama/blob/llama_v1/FAQ.md
</table>

This new paradigm of adapting the behavior of a Pre-Trained model *without* further Fine-Tuning
is called ["Pre-train, Prompt, Predict"](https://arxiv.org/pdf/2107.13586.pdf).

In this module, we explore the "art" of constructing prompts to elicit new behaviors.

# Prompt engineering

You see how the behavior of the LLM can be affected by the exact form of the prompt.
- the number of exemplars

It is also the case that slightly changing the words (and order of words) affects behavior.

There is a whole literature on creating successful prompts:
- this [website](https://www.promptingguide.ai/) gives good advice in crafting a prompt to meet your expectations
- this [paper](https://arxiv.org/pdf/2107.13586.pdf) is a systematic survey
- OpenAI provides [helpful examples](https://platform.openai.com/examples) for prompting.
- [See Appendix G](https://arxiv.org/pdf/2005.14165.pdf#page=51) (pages 50+) for examples of prompts on creating prompts to solve some standard tasks

We won't examine all of the useful tricks.

But one method has proven to be exceptionally successful: Chain of Thought Prompting.

## Chain of thought prompting

[Paper: Chain of thought prompting](https://arxiv.org/pdf/2201.11903.pdf)

In school, students are often tasked with solving problems involving multiple steps.

LLM's are better at multi-step reasoning tasks when they have been conditioned to answer step by step.

We call this *chain of thought (CoT)* prompting

The **exemplars** used in CoT prompting
- demonstrate step by step reasoning in the expected output

We can see the difference in the exemplar's "Example Output" section
- using "Standard Prompting" (on the left)
- versus using "CoT Prompting" (on the right)
    - the first exemplar (example) demonstrates step-by-step reasoning

<table>
    <center><strong>Chain of Thought Prompting</strong></center>
    <tr>
        <img src="images/cot_prompt_example.png" width=80%>
    </tr>
    
    Attribution: https://arxiv.org/pdf/2201.11903.pdf
</table>


How does this apply to the case of *zero* exemplars (zero-shot learning) ?

It turns out that step by step reasoning can be elicited
- Just by adding the phrase ["Let's think step by step"](https://arxiv.org/pdf/2205.11916.pdf) to the end of the query

Let's see an example.

Let's ask ChatGPT to solve a multi-step reasoning problem in a zero-shot setting.

As you can see: it comes close, by produces an incorrect answer.

<img src="images/cot_prompt_no_step_by_step.png">

The reasoning logic is correct, but the math is wrong.

Now, let's run the same query but append a request to answer step-by-step.

<img src="images/cot_prompt_step_by_step.png">

The "LETS THINK STEP BY STEP" seems to condition the model into getting the math correct
- Without CoT: answers comes before reasoning
- With CoT: reasoning precedes answer

It is the conditioning of providing the reasoning before the answer that seems to improve the behavior.

# Using zero-shot to create new applications

With a little cleverness, one can almost trivially create a new application using a LLM in zero-shot mode
- create the prefix of a prompt describing the task
- append the user input to the prefix to complete the prompt

Here we use [`ChatGPT`](https://chat.openai.com/chat) to create an app that summarizes a conversation
- we create a prompt with a "place-holder" (in braces `{..}`) for user input

`prompt = Summarize the following conversation: {user input}`

<img src="images/chatgpt_summarize_conversation_example.png" width=80%>

Here we use ChatGPT as a programming assistant

`prompt = Write a Python function that does the following: {task description}`

<img src="images/chatgpt_program_generation_example.png" width=80%>

## Some more, creative examples
- [Spreadsheet add-in to perform lookups](https://twitter.com/pavtalk/status/1285410751092416513)
- [Generate a web page from a description](https://twitter.com/sharifshameem/status/1283322990625607681)

References found in: http://ai.stanford.edu/blog/understanding-incontext/

# How is zero-shot learning possible ? Some theories

**Theory 1**

- The training set contains explicit instances of these out of sample tasks

**Theory 2**

- The super-large training sets  contain *implicit* instances of these out of sample tasks
    - For example: an English-language article quoting a French speaker in French with English translation

One thing that jumps out from the graph:
- Bigger models are more likely to exhibit meta-learning

**Theory 3**

The training sets are so big that the model "learns" to create groups of examples with a common theme
- Even with the large number of parameters, the model capacity does not suffice for example memorization


Another thing to consider
- The behavior of an RNN depends on *all* previous inputs
    - It has memory (latent state, etc.)
    
So Few Shot Learning may work by "priming" the memory with parameters for a specific task

# Social concerns

The team behind GPT is very concerned about potential misuse of Language Models.

To illustrate, they conducted an experiment in having a Language Model construct news articles
- Select title/subtitle of a genuine news article
- Have the Language Model complete the article from the title/subtitle
- Show humans the genuine and generated articles and ask them to judge whether the article was written by a human

<table>
    <tr>
        <th><center>Human accuracy in detecting model generated news articles</center></th>
    </tr>
    <tr>
        <td><img src="images/LM_GPT_model_generated_news.png" width=80%></td>
    </tr>
    <tr>
        <td><center>Picture from: https://arxiv.org/pdf/2005.14165.pdf</center></td>
    </tr>   
</table>

The bars show the range of accuracy across the 80 human judges.

- 86% accuracy detecting articles created by a really bad model (the control)
- 50% accuracy detecting articles created by the biggest models

It seems that humans might have difficulty distinguishing between genuine and generated articles.

The fear is that Language Models can be used
- to mislead
- to create offensive speech

## Harmful behavior of the Pre-Trained LLM

Another important concern is that LLM's can exhibit bias
- we saw similar behavior with embeddings
- the bias is statistical: a property of the training data

Any model based on the Pre-Trained model might inherit this bias.

A recent trend in model development
- is to probe the model for harmful behavior

Models are now often described via a [Model card](https://arxiv.org/abs/1810.03993)
- describing both the technical properties
- and the social aspects (biases)

[Here](https://huggingface.co/openai-gpt) is an example of a model card.

In [2]:
print("Done")

Done
